## Metadata and Record Linkage

In this tutorial, we will show some examples for creating metadata given a tabular data. There are a lot of Python tools which can help to extract metadata from the data.

Secondly, we will show how to make record linkage given two tables. In these tables, some records are duplicated in other tables. 

### 1. Metadata
We will use the Python tool `frictionless` to infer the metadata from a table.

In [2]:
from pprint import pprint
from frictionless import describe, Resource, Package    
import json

The object `describe` returns a `TableResource`, which contains metadata information

In [4]:
resource = describe('datasets/dataset_D_org_1000.csv')

In [5]:
pprint(resource.to_dict())

{'encoding': 'utf-8',
 'format': 'csv',
 'mediatype': 'text/csv',
 'name': 'dataset_d_org_1000',
 'path': 'datasets/dataset_D_org_1000.csv',
 'schema': {'fields': [{'name': 'rec_id', 'type': 'integer'},
                       {'name': 'given_name', 'type': 'string'},
                       {'name': 'surname', 'type': 'string'},
                       {'name': 'street_number', 'type': 'integer'},
                       {'name': 'address_1', 'type': 'string'},
                       {'name': 'address_2', 'type': 'string'},
                       {'name': 'suburb', 'type': 'string'},
                       {'name': 'postcode', 'type': 'integer'},
                       {'name': 'state', 'type': 'string'},
                       {'name': 'date_of_birth', 'type': 'integer'},
                       {'name': 'age', 'type': 'integer'},
                       {'name': 'phone_number', 'type': 'string'},
                       {'name': 'soc_sec_id', 'type': 'integer'},
                       {'na

We can add some fields like `description` in the metadata in this example, which describes the data table. Any other fields can also be added.

In [6]:
resource.description='This is a dataset of 1000 organisation records with metadata.'
resource.title='Organisation Records with Metadata'


In [7]:
pprint(resource)

{'name': 'dataset_d_org_1000',
 'type': 'table',
 'title': 'Organisation Records with Metadata',
 'description': 'This is a dataset of 1000 organisation records with metadata.',
 'path': 'datasets/dataset_D_org_1000.csv',
 'scheme': 'file',
 'format': 'csv',
 'mediatype': 'text/csv',
 'encoding': 'utf-8',
 'schema': {'fields': [{'name': 'rec_id', 'type': 'integer'},
                       {'name': 'given_name', 'type': 'string'},
                       {'name': 'surname', 'type': 'string'},
                       {'name': 'street_number', 'type': 'integer'},
                       {'name': 'address_1', 'type': 'string'},
                       {'name': 'address_2', 'type': 'string'},
                       {'name': 'suburb', 'type': 'string'},
                       {'name': 'postcode', 'type': 'integer'},
                       {'name': 'state', 'type': 'string'},
                       {'name': 'date_of_birth', 'type': 'integer'},
                       {'name': 'age', 'type': 'integ

The metadata can be imported to `JSON` file.

In [8]:
discriptor = resource.to_descriptor()

# write descriptor to json file
with open('dataset_D_org_1000_descriptor.json', 'w') as f:
    json.dump(discriptor, f, indent=4)

There are two tables in this example. In the following, both metadata can be inferred.

In [11]:
dataset_org = Resource('datasets/dataset_D_org_1000.csv')
dataset_dup = Resource('datasets/dataset_D_dup_1000.csv')

In [12]:
# infer metadata for both tables
dataset_org.infer()
dataset_dup.infer()

Data descriptions can be added to the metadata as shown previously. 
The collection of the data resources can be created

In [13]:
package = Package(resources=[dataset_org, dataset_dup])

Then infer metadata from package-level

In [14]:
package.infer()

Then save the metadata to a JSON file

In [15]:
package.to_json('datasets/dataset_1000_metadata.json')

'{\n  "resources": [\n    {\n      "name": "dataset_d_org_1000",\n      "type": "table",\n      "path": "datasets/dataset_D_org_1000.csv",\n      "scheme": "file",\n      "format": "csv",\n      "mediatype": "text/csv",\n      "encoding": "utf-8",\n      "schema": {\n        "fields": [\n          {\n            "name": "rec_id",\n            "type": "integer"\n          },\n          {\n            "name": "given_name",\n            "type": "string"\n          },\n          {\n            "name": "surname",\n            "type": "string"\n          },\n          {\n            "name": "street_number",\n            "type": "integer"\n          },\n          {\n            "name": "address_1",\n            "type": "string"\n          },\n          {\n            "name": "address_2",\n            "type": "string"\n          },\n          {\n            "name": "suburb",\n            "type": "string"\n          },\n          {\n            "name": "postcode",\n            "type": "integer"

### 2. Record Linkage

This example shows how to find duplicated records from two data tables. The code is adapted from the Python Record Linkage Toolkit tutorial: https://recordlinkage.readthedocs.io/en/latest/guides/data_deduplication.html

The way to link the data record is to choose which attributes to be used for linking them.

Firstly, the package is imported

In [17]:
import recordlinkage
import pandas as pd

The datasets will be read using `pandas.DataFrame`

In [18]:
org_df = pd.read_csv('datasets/dataset_D_org_1000.csv')
dup_df = pd.read_csv('datasets/dataset_D_dup_1000.csv')

In [19]:
org_df

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id,blocking_number
0,3040,lachlan,curtis,19.0,dale place,NaN,ungarie,3028.0,qld,19870906.0,24.0,07 79183195,6224611,2
1,3480,savannah,warburton,17.0,wambo close,NaN,collarenebri,2112.0,NaN,19690110.0,20.0,03 42548839,8612132,4
2,4660,caitlin,panawo,266.0,lidgett place,NaN,ingham,3028.0,NaN,19231017.0,33.0,04 02919656,1269996,8
3,1620,jacynta,kessner,27.0,tauchert street,NaN,wahroonga,5063.0,qld,19370624.0,27.0,08 61313982,8731381,7
4,720,jesse,sutherland,3.0,light street,NaN,adelong,2428.0,vic,19110305.0,19.0,02 47460338,1942604,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,4760,william,britcher,370.0,colebatch place,kilmarnock park,sinnamon park,6428.0,qld,19740314.0,NaN,07 40730324,2002052,4
496,1970,dylan,jones,66.0,pudney street,NaN,dover gardens,2620.0,tas,NaN,32.0,03 54119423,1222346,9
497,340,paris,andrews,63.0,oldfield circuit,NaN,lemon tree passage,2099.0,nsw,19690225.0,25.0,07 49098520,4061703,9
498,2080,kane,prior,31.0,britten-jones drive,NaN,lucas heights,3088.0,nsw,19810128.0,25.0,04 23825360,4724147,9


#### Make record pairs

The first task is to compare each record in DataFrame `org_df` with all other records in `dup_df`. Each record in `org_df` will be a pair with each one in `dup_df`. The number of pairs will be `len(org_df)*len(dup_df)`. 

In the Record Linkage Tool, the process of making record pairs is called `indexing`. With the recordlinkage module, indexing is easy. First, load the `recordlinkage.Index` class and call the `.full` method. This object generates a full index on a `.index(...)` call. In case of deduplication of a single dataframe, one dataframe is sufficient as argument. 

In [20]:
indexer = recordlinkage.Index()
indexer.full()  # use full indexing method
candidate_pairs = indexer.index(org_df, dup_df)

In [21]:
print(len(org_df),len(dup_df),len(candidate_pairs))

500 500 250000


Most of these pairs are not really identical or belong to the same record in the data.

One of the most well known indexing methods is named `blocking`. This method includes only record pairs that are identical on one or more stored attributes of the person (or entity in general). The blocking method can be used in the `recordlinkage` module. This indicates that duplications can be exact duplication or near duplication.

The following example shows to use `given_name` and `surname` as the blocking variable. The number of candidate pairs are significantly reduced.

In [22]:
indexer = recordlinkage.Index()
indexer.block(['given_name', 'surname'])  # use blocking method on given_name and surname
candidate_pairs = indexer.index(org_df, dup_df)
len(candidate_pairs)

361

#### Compare records

Each record pair is a candidate match. To classify the candidate record pairs into matches and non-matches, compare the records on all attributes both records have in common. The `recordlinkage` module has a class named `Compare`. This class is used to compare the records. The following code shows how to compare attributes.

In [23]:
compare_cl = recordlinkage.Compare()
compare_cl.exact("given_name", "given_name", label="given_name")
compare_cl.string(
    "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
)
compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
compare_cl.exact("suburb", "suburb", label="suburb")
compare_cl.exact("state", "state", label="state")
compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")
features = compare_cl.compute(candidate_pairs, org_df, dup_df)


The comparing of record pairs starts when the `compute` method is called. All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.

In [24]:
features

,,given_name,surname,date_of_birth,suburb,state,address_1
1,454,1,1.0,1,1,0,1.0
2,326,1,1.0,1,1,0,1.0
3,2,1,1.0,0,1,1,1.0
4,9,1,1.0,1,0,1,1.0
5,16,1,1.0,1,1,1,1.0
...,...,...,...,...,...,...,...
494,58,1,1.0,1,1,1,1.0
495,185,1,1.0,1,1,1,1.0
497,249,1,1.0,1,0,1,1.0
498,322,1,1.0,1,1,1,1.0


In [25]:
# this shows the statistics of the features
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,361.0,361.0,361.000000,361.000000,361.000000,361.000000
mean,1.0,1.0,0.750693,0.886427,0.817175,0.947368
std,0.0,0.0,0.433213,0.317733,0.387060,0.223607
min,1.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.0,1.000000,1.000000,1.000000,1.000000
50%,1.0,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.0,1.0,1.000000,1.000000,1.000000,1.000000
max,1.0,1.0,1.000000,1.000000,1.000000,1.000000


The last step is to decide which records belong to the same person. In this example, we keep it simple:

In [26]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0    176
5.0    156
4.0     27
3.0      2
Name: count, dtype: int64

This indicates that there 176 records which are exact duplications according to these six features. There are 156 records, which matches on five features.